In [9]:
# test: Query a form from fulcrum (Preventive Maintenance) and publish to postgresql
# import packages
import pandas as pd
import numpy as np
import knackpy as kp
import fulcrum as fc
import requests
import pdb
from datetime import datetime, timedelta
# import credentials
from config.secrets import *
from tdutils.pgrestutil import Postgrest
import json

In [10]:
pd.options.display.max_columns = 2000

In [11]:
form_id = "44359e32-1a7f-41bd-b53e-3ebc039bd21a"
key = FULCRUM_CRED.get("api_key")

In [12]:
pgrest = Postgrest("http://transportation-data-test.austintexas.io/signal_pms", auth=JOB_DB_API_TOKEN_test)

In [13]:
"""Summary

Attributes:
    form_id (str): Description
    key (TYPE): Description
    pgrest (TYPE): Description
"""
# test: Query a form from fulcrum (Preventive Maintenance) and publish to postgresql
# import packages
import pandas as pd
import numpy as np
import knackpy as kp
import fulcrum as fc
import requests
import pdb
from datetime import datetime, timedelta
# import credentials
from config.secrets import *

from tdutils.pgrestutil import Postgrest


form_id = "44359e32-1a7f-41bd-b53e-3ebc039bd21a"
key = FULCRUM_CRED.get("api_key")

# create postgrest instance
pgrest = Postgrest("http://transportation-data-test.austintexas.io/signal_pms", auth=JOB_DB_API_TOKEN_test)


# test: Query a form from fulcrum (Preventive Maintenance)

def recur_dict(col_names, elements):
    """Summary
    
    Args:
        col_names (TYPE): Description
        elements (TYPE): Description
    
    Returns:
        TYPE: Description
    """


    # print(type(elements))

    for element in elements:

        if type(element) == dict:
            col_names[element.get("key")] = element.get("data_name")

            for key, value in element.items():
                if type(value) == list:
                    recur_dict(col_names, value)

    return col_names



def get_col_names(form_id):
    """Summary
    get a dictionary with {"key": "label"} pair for all columns in a data
    base
    
    
    
    Args:
        form_id (TYPE): Description
    
    
    
    Returns:
        TYPE: Description
    
    
    
    """

    form = fulcrum.forms.find(form_id)
    elements = form.get("form").get("elements")

    col_names = {}

    col_names = recur_dict(col_names, elements)

    return col_names

def get_records(form_id):
    """Summary
    
    Args:
        form_id (TYPE): Description
    
    Returns:
        TYPE: Description
    """
    # initiate a dataframe

    records_dirty = fulcrum.records.search(url_params = {"form_id":form_id})

    records = pd.DataFrame()



    for record in records_dirty["records"]:
        form_values = record["form_values"]
        for key, value in form_values.items():
            # print(value)
            if type(value) == dict and "choice_values" in value:
                value = value["choice_values"]
                if type(value) == list and len(value) == 2:
                    if key == "fce3":
                        form_values[key] = value[1]
                
            if type(value) == list and len(value) == 1:
                form_values[key] = value[0]
            
                
        form_values["_server_updated_at"] = record["created_at"]
        form_values["_record_id"] = record["id"]
        new_row = pd.DataFrame([form_values], columns=form_values.keys())
    #     print(new_row)
        records = pd.concat([new_row, records], axis =0, sort=False).reset_index(drop=True)

    return records

def interpret_col_name(records):
    """Summary
    
    Args:
        records (TYPE): Description
    
    Returns:
        TYPE: Description
    """
    return records.rename(columns=col_names)


def clean_pm(records):
    """Summary
    
    Args:
        records (TYPE): Description
    
    Returns:
        TYPE: Description
    """

    
    df = records.copy()
    
#     print(df)

    df["signal_id"] = df["signal"].str.split("|").str[0]

    # rename record_id to fulcrum ID
    df = df.rename(columns={'_record_id': 'fulcrum_id'})

    df["_server_updated_at"] = df["_server_updated_at"].str.strip("CDT")
    df["pm_completed_datetime"] = pd.to_datetime(df["_server_updated_at"], format = "%Y-%m-%d %H:%M:%S")
    df["pm_completed_date"] = df["pm_completed_datetime"].dt.date
    
#     print(df["technicians"])

    df["pm_completed_by"] = df["technicians"]#.str.split(",").str[1]
    df["modified_current"] = datetime.now().replace(microsecond=0).isoformat(" ")

    df["modified_current"] =pd.to_datetime(df["modified_current"], format = "%Y-%m-%d %H:%M:%S")

    df["modified_date"] = df["modified_current"] + timedelta(minutes = 20)

    cleaned_record = df[["signal_id", "fulcrum_id", "pm_completed_date", "modified_date", "pm_completed_by"]]

    return cleaned_record

def get_pgrest_records():
    # the datetime converstin for modified_date is not right. The time part are missing

    results = pgrest.select('')
    results = pd.DataFrame(results)
    results["modified_date"] = pd.to_datetime(results["modified_date"], format='%Y-%m-%dT%H:%M:%S')
#     results["modified_date"] = results["modified_date"](format="'%Y-%m-%dT%H:%M:%S'")
    results["pm_completed_date"] = pd.to_datetime(results["pm_completed_date"])
    
    return results

def get_most_recent():
    """Summary

    
    Returns:
        TYPE: Description
    """
    results = pgrest.select('fulcrum_id=eq.abc123')

    return results


def main():
    """Summary
    """
    
    
    
    pass



In [14]:
fulcrum = fc.Fulcrum(key = key)
forms = fulcrum.forms.search(url_params={'id': form_id})
col_names = get_col_names(form_id)
records = get_records(form_id)

records = interpret_col_name(records)

fulcrum_records = clean_pm(records)

print(records)

                                              signal         preemption  \
0                  952 |  CAMERON RD / CROSS PARK DR           Railroad   
1                       289 |  8TH ST / RED RIVER ST  Emergency Vehicle   
2         238 |  CESAR CHAVEZ ST / WALTER SEAHOLM DR      No Preemption   
3        229 |  CESAR CHAVEZ ST / SANDRA MURAIDA WAY      No Preemption   
4                  440 |  NORTHLAND DR / BALCONES DR      No Preemption   
5                675 |  ANDERSON MILL RD / PARMER LN      No Preemption   
6                  673 |  PARMER LN / AMBERGLEN BLVD      No Preemption   
7                       674 |  PARMER LN / TAMAYO DR      No Preemption   
8               172 |  AIRPORT BLVD / KOENIG LN SVRD      No Preemption   
9           4050 |  QUINLAN PARK RD / CANYON GLEN DR      No Preemption   
10       914 |  QUINLAN PARK RD / STEINER RANCH BLVD      No Preemption   
11       948 |  QUINLAN PARK RD / STEINER RANCH BLVD      No Preemption   
12        923 |  QUINLAN 

In [18]:
fulcrum_records.head()

,signal_id,fulcrum_id,pm_completed_date,modified_date,pm_completed_by
0,952,341cf032-e684-4319-b323-47f5dce43081,2018-11-20,2018-11-26 16:17:07,john.clary@austintexas.gov
1,289,ff4687c8-b044-4f7b-aabb-b0e267a214be,2018-09-21,2018-11-26 16:17:07,toby.turner@austintexas.gov
2,238,119530f5-e0ce-4a3d-9d5f-39a78c39f71f,2018-09-06,2018-11-26 16:17:07,toby.turner@austintexas.gov
3,229,c5b17f94-6e61-4c7a-84b6-de51ac49774c,2018-09-06,2018-11-26 16:17:07,toby.turner@austintexas.gov
4,440,acb8724f-f499-43eb-81f5-da3566539418,2018-09-04,2018-11-26 16:17:07,toby.turner@austintexas.gov


In [20]:
def prepare_payload(fulcrum_records, pgrest_records):
    # compare the modified date and fulcrum id in fulcrum records and 
    # in postgrest record. 

    payload = fulcrum_records[~fulcrum_records['fulcrum_id'].isin(pgrest_records['fulcrum_id'])]

    return payload

In [21]:
payload = prepare_payload(fulcrum_records, pgrest_records)

NameError: name 'pgrest_records' is not defined

In [22]:
def upload_pgrest(payload):
    
    payload = payload.to_dict(orient='index')
    payload = json.dumps(payload, indent=4, sort_keys=True, default=str)

    upload_result = pgrest.upsert(payload)
    
    return upload_result

In [ ]:
upload_pgrest(payload)



In [24]:
records.sample(10)



,signal,preemption,cameras,leave_rat_bait_in_all_pull_boxes_cabinet,camera_lenses,conflict_monitor,arrival_time,technicians,departure_time,railroad_preemption,has_detection,dc_isolators,screw_terminals_tightness,flashers_flash_plugs_flash_relays,load_switches,fan_operation,cabinet_interior,signal_colors_alignment,type_of_ped_buttons,ped_buttons,aps_beepers,controller_time,street_name_signs,ups,signal_head_obstructions,backup_data_key,lube_cabinet_door_lock,graffiti,intersection_condition,follow_up_needed,intersection_layout,_server_updated_at,_record_id,emergency_vehicle_preemption,detection,follow_up_description,other_images,graffiti_description
19,808 | BRAKER LN / RESEARCH BLVD SVRD,No Preemption,OK,yes,Cleaned,{'other_values': ['ok and programmed 172.16.1...,05:00,toby.turner@austintexas.gov,06:29,NaN,yes,OK,OK,OK,OK,OK,Cleaned,OK,Bulldog,OK,No APS Beepers,OK,OK,{'other_values': ['Ups is dead needs batteries...,No Obstruction,no,yes,no,GOOD,yes,{'photo_id': '1f75306a-bdbf-476d-8d3b-47460e92...,2018-08-23T12:07:32Z,6614833d-c98a-4972-a908-668407bfb6e2,NaN,"[{'updated_at': '1535021257', 'created_locatio...",1.South bound loop wire bad road way and pull ...,{'photo_id': 'f3e1b36a-1f35-48f0-b1aa-16e02506...,NaN
29,707 | US 183 HWY SVRD / PECAN PARK BLVD,No Preemption,No Camera,yes,NaN,"{'other_values': ['replaced and programmed '],...",10:22,toby.turner@austintexas.gov,11:39,NaN,no,OK,OK,OK,OK,OK,Cleaned,OK,Bulldog,OK,No APS Beepers,OK,OK,OK,No Obstruction,yes,yes,NaN,GOOD,no,{'photo_id': '6a9d1242-253a-414e-b133-41dfc83b...,2018-08-15T16:26:30Z,2d2ae1cc-91c5-4c9f-8578-acd9f2b52d8d,NaN,NaN,NaN,NaN,NaN
24,721 | MC NEIL DR / 183 HWY SVRD,Emergency Vehicle,OK,yes,Cleaned,Replaced,09:47,toby.turner@austintexas.gov,11:23,NaN,yes,OK,OK,OK,OK,OK,Cleaned,OK,Bulldog,OK,No APS Beepers,OK,OK,No UPS,No Obstruction,yes,yes,no,GOOD,yes,{'photo_id': '09b33e1e-052f-4e24-a19b-86d999f3...,2018-08-20T16:36:50Z,06981c0b-0db1-446e-b8b0-7c9e0a30a63d,"{'other_values': ['not working '], 'choice_val...","{'updated_at': '1534783002', 'created_location...",Preemption,{'photo_id': '341c9e91-db8a-4fbb-808f-a8c5509b...,NaN
5,675 | ANDERSON MILL RD / PARMER LN,No Preemption,OK,yes,Cleaned,OK,07:22,toby.turner@austintexas.gov,07:45,NaN,yes,OK,OK,OK,OK,OK,{'other_values': ['cabinet was cleaned but has...,OK,Push-Button,OK,No APS Beepers,OK,{'other_values': ['north bound Parmer missing ...,{'other_values': ['Ups is dead needs batteries...,No Obstruction,yes,yes,no,GOOD,yes,{'photo_id': 'd104f3c5-15a5-4666-9141-d3f0499e...,2018-09-04T12:57:00Z,c2251729-2927-4556-b558-4cc2178a976f,NaN,"[{'updated_at': '1536063808', 'created_locatio...",1.Supervisor needs to check if cabinet is in n...,{'photo_id': '44fdb071-0b6c-482d-b738-7538ef6c...,NaN
40,1017 | ANDERSON MILL RD / CASHELL WOOD DR,No Preemption,No Camera,yes,NaN,Replaced,07:03,toby.turner@austintexas.gov,08:15,NaN,yes,OK,OK,OK,OK,OK,Cleaned,OK,APS,OK,OK,"{'other_values': ['d4'], 'choice_values': []}",OK,No UPS,No Obstruction,no,yes,NaN,EXCELLENT,no,{'photo_id': '4376342c-5f9a-4312-87c3-d1f1be4a...,2018-08-07T14:15:16Z,a91eae80-1723-41bf-9828-e51ef8525648,NaN,"[{'updated_at': '1533643768', 'created_locatio...",NaN,{'photo_id': '0f7fdb70-4153-4925-b36a-32accccc...,NaN
18,717 | OHLEN RD / RESEARCH BLVD SVRD,No Preemption,OK,yes,Cleaned,{'other_values': ['could not change fya monito...,05:39,toby.turner@austintexas.gov,06:37,NaN,no,OK,OK,OK,OK,OK,Cleaned,OK,Bulldog,OK,No APS Beepers,OK,OK,{'other_values': ['Ups is dead needs batteries...,No Obstruction,yes,yes,yes,GOOD,yes,{'photo_id': 'db04de88-cf7f-424f-adb2-79af3fa3...,2018-08-24T11:37:22Z,44b106b6-5068-421a-9f18-2ce6036a3bfe,NaN,NaN,1. Ups is dead needs batteries \n2. Graffiti o...,{'photo_id': '1aacb93e-54e5-45b2-b8d7-8e3dbe1d...,On pole south east pole on ohlen rd
27,604 | LAKE CREEK PKWY / 183 HWY SVRD,No Preemption,No Camera,yes,NaN,Replaced,12:57,toby.turner@austintexas.gov,13:25,NaN,yes,OK,OK,OK,OK,OK,Cleaned,{'other_values': ['replaced 2 green led east b